<a href="https://colab.research.google.com/github/10100111/Display-of-HW1/blob/main/12_Ultra_Pro_%D0%A1%D0%B2%D0%B5%D1%80%D1%82%D0%BE%D1%87%D0%BD%D1%8B%D0%B5_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D0%BD%D0%BD%D1%8B%D0%B5_%D1%81%D0%B5%D1%82%D0%B8_(%D0%A3%D0%BD%D0%B8%D0%B2%D0%B5%D1%80%D1%81%D0%B8%D1%82%D0%B5%D1%82_%D0%B8%D1%81%D0%BA%D1%83%D1%81%D1%81%D1%82%D0%B2%D0%B5%D0%BD%D0%BD%D0%BE%D0%B3%D0%BE_%D0%B8%D0%BD%D1%82%D0%B5%D0%BB%D0%BB%D0%B5%D0%BA%D1%82%D0%B0).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow.keras
from tensorflow.keras.models import Sequential, Model  # Сеть прямого распространения
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, BatchNormalization, GlobalMaxPooling2D, \
    GlobalAveragePooling2D, AveragePooling2D, Flatten, ZeroPadding2D, Activation

from tensorflow.keras import utils  # Используем дял to_categoricall
from tensorflow.keras.preprocessing import image  # Для отрисовки изображений
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
import numpy as np  # Библиотека работы с массивами
import matplotlib.pyplot as plt  # Для отрисовки графиков
from PIL import Image  # Для отрисовки изображений
from tensorflow.keras.optimizers import Adam

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import gdown
# Загрузка zip-архива с датасетом из облака на диск виртуальной машины colab
gdown.download('https://storage.googleapis.com/datasets_ai/Base/5_Conv/middle_fmr.zip', None, quiet=True)

'middle_fmr.zip'

In [ ]:
!unzip -q "middle_fmr.zip" -d /content/cars #Указываем путь к базе в Google Drive

In [ ]:
train_path = '/content/cars' #Папка с папками картинок, рассортированных по категориям
batch_size = 80 #Размер выборки
img_width = 96 #Ширина изображения
img_height = 54 #Высота изображения

In [ ]:
#Генератор изображений для обучения
train_datagen = ImageDataGenerator(
    rescale=1. / 255, #Значения цвета меняем на дробные показания
    rotation_range=4,  #Поворачиваем изображения при генерации выборки в диапазоне на 4 градуса
    width_shift_range=0.04, #Диапазон сдвига по ширине 0.04 (можно от -1 до 1)
    height_shift_range=0.04, #Диапазон сдвига по высоте в долях от общей высоты 0.04 (можно от -1 до 1)
    zoom_range=[0.8, 1.0], #Диапазон случайного увеличения (зумирование)
    shear_range=0.04, #Угол сдвига против часовой стрелки в градусах
    brightness_range=[0.3, 1.0], #яркость_диапазона   
    validation_split=0.2
)
# обучающая выборка
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_datagen = ImageDataGenerator(
    rescale=1. / 255, #Значения цвета меняем на дробные показания
    validation_split=0.2 #Указываем размер валидационной выборки
)

# Проверочная выборка
validation_generator = validation_datagen.flow_from_directory(
    train_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 3427 images belonging to 3 classes.
Found 684 images belonging to 3 classes.


In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(img_width, img_height, 3)))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(len(train_generator.class_indices), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 96, 54, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 96, 54, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 48, 27, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 48, 27, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 27, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 13, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 13, 64)        3

In [ ]:
epochs = 40 

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs,
    verbose=1
)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
42/42 [==============================] - 41s 308ms/step - loss: 0.9953 - accuracy: 0.4798 - val_loss: 0.9405 - val_accuracy: 0.5359
Epoch 2/40
42/42 [==============================] - 12s 296ms/step - loss: 0.9125 - accuracy: 0.5369 - val_loss: 0.9613 - val_accuracy: 0.5016
Epoch 3/40
42/42 [==============================] - 12s 296ms/step - loss: 0.9088 - accuracy: 0.5596 - val_loss: 0.9662 - val_accuracy: 0.5250
Epoch 4/40
42/42 [==============================] - 13s 298ms/step - loss: 0.8777 - accuracy: 0.5919 - val_loss: 0.8919 - val_accuracy: 0.5750
Epoch 5/40
42/42 [==============================] - 12s 291ms/step - loss: 0.8671 - accuracy: 0.5937 - val_loss: 0.8779 - val_accuracy: 0.5828
Epoch 6/40
42/42 [==============================] - 12s 290ms/step - loss: 0.8631 - accuracy: 0.6005 - val_loss: 0.8694 - val_accuracy: 0.6047
Epoch 7/40
42/42 [==============================] - 12s 290ms/step - loss: 0.8166 - accuracy: 0.6256 - val_loss: 0.8544 - val_accuracy: 0.6391

In [ ]:
model.save('cars.h5')